# Sales Forecast Model V2


In [1]:
# Import required libraries and setup environment
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error
import os
import sys

# Optional: Optuna for hyperparameter optimization
try:
    import optuna
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False
    print("Warning: Optuna not available. Install with: pip install optuna")


In [2]:
# Define the main SalesForecastModelV2 class with initialization
class SalesForecastModelV2:
    def __init__(self, data_path="data/"):
        self.data_path = data_path
        self.transactions = None
        self.products = None
        self.stores = None
        self.model = None
        self.label_encoders = {}
        self.validation_metrics = {}
        self.best_params = None
        self.target_log_transformed = True


In [3]:
# Load data from parquet files and identify data types
def load_data(self):
    print("Loading data...")
    parquet_files = [f for f in os.listdir(self.data_path) if f.endswith('.parquet')]
    print(f"Found {len(parquet_files)} parquet files")
    
    for file in parquet_files:
        df = pd.read_parquet(os.path.join(self.data_path, file))
        print(f"{file}: Shape {df.shape}")
        
        if 'internal_store_id' in df.columns and 'quantity' in df.columns:
            self.transactions = df
            print("-> Identified as TRANSACTIONS data")
        elif 'produto' in df.columns and 'categoria' in df.columns:
            self.products = df
            print("-> Identified as PRODUCTS data")
        elif 'pdv' in df.columns and 'premise' in df.columns:
            self.stores = df
            print("-> Identified as STORES data")
    
    print(f"Data loaded successfully:")
    print(f"- Transactions: {self.transactions.shape[0]:,} rows")
    print(f"- Products: {self.products.shape[0]:,} rows")
    print(f"- Stores: {self.stores.shape[0]:,} rows")

SalesForecastModelV2.load_data = load_data


In [4]:
# Clean data by removing nulls, outliers, and filtering to 2022 data
def cleanse_data(self):
    print("\nCleansing data...")
    
    initial_rows = len(self.transactions)
    self.transactions = self.transactions.dropna(subset=['internal_store_id', 'internal_product_id', 'quantity', 'transaction_date'])
    print(f"Removed {initial_rows - len(self.transactions):,} rows with null values")
    
    self.transactions = self.transactions[self.transactions['quantity'] > 0]
    print(f"Kept {len(self.transactions):,} rows with positive quantities")
    
    # Remove extreme value outliers (likely data errors)
    print("Removing extreme value outliers...")
    initial_rows = len(self.transactions)
    
    # Calculate value per unit to detect unrealistic transactions
    value_per_unit = self.transactions['gross_value'] / self.transactions['quantity']
    q01 = value_per_unit.quantile(0.005)
    q99 = value_per_unit.quantile(0.995)
    
    # Remove transactions with extreme value per unit
    valid_value_mask = (value_per_unit >= q01) & (value_per_unit <= q99)
    self.transactions = self.transactions[valid_value_mask]
    
    # Also cap extreme quantities (likely bulk orders or errors)
    quantity_q99 = self.transactions['quantity'].quantile(0.995)
    extreme_qty_mask = self.transactions['quantity'] <= quantity_q99
    self.transactions = self.transactions[extreme_qty_mask]
    
    print(f"Removed {initial_rows - len(self.transactions):,} outlier transactions ({((initial_rows - len(self.transactions))/initial_rows)*100:.2f}%)")
    
    self.transactions[['transaction_date', 'reference_date']] = self.transactions[['transaction_date', 'reference_date']].apply(pd.to_datetime)
    
    self.transactions = self.transactions[
        (self.transactions['transaction_date'].dt.year == 2022)
    ]
    print(f"Filtered to 2022 data: {len(self.transactions):,} rows")
    
    print("Cleaning products and stores...")
    self.products['descricao'] = self.products['descricao'].fillna('Unknown')
    self.products['categoria'] = self.products['categoria'].fillna('Other')
    self.products['marca'] = self.products['marca'].fillna('Unknown')
    
    self.stores['categoria_pdv'] = self.stores['categoria_pdv'].fillna('Other')
    self.stores['premise'] = self.stores['premise'].fillna('Unknown')

SalesForecastModelV2.cleanse_data = cleanse_data


In [5]:
# Merge transactions with product and store data
def merge_data(self):
    print("\nMerging data...")
    
    merged_data = self.transactions.merge(
        self.products, 
        left_on='internal_product_id', 
        right_on='produto', 
        how='left'
    )
    print(f"After product merge: {len(merged_data):,} rows")
    
    merged_data = merged_data.merge(
        self.stores,
        left_on='internal_store_id',
        right_on='pdv',
        how='left'
    )
    print(f"After store merge: {len(merged_data):,} rows")
    
    self.merged_data = merged_data
    print("Data merge completed")

SalesForecastModelV2.merge_data = merge_data


In [6]:
# Create weekly aggregations and temporal features
def create_weekly_aggregations(self):
    print("\nCreating weekly aggregations...")
    
    dt_info = self.merged_data['transaction_date'].dt
    self.merged_data['year'] = dt_info.year
    self.merged_data['week'] = dt_info.isocalendar().week
    self.merged_data['month'] = dt_info.month
    self.merged_data['quarter'] = dt_info.quarter
    self.merged_data['year_week'] = self.merged_data['year'].astype(str) + '_' + self.merged_data['week'].astype(str).str.zfill(2)
    
    weekly_data = self.merged_data.groupby([
        'year_week', 'week', 'month', 'quarter', 'internal_store_id', 'internal_product_id',
        'categoria', 'marca', 'premise', 'categoria_pdv'
    ]).agg({
        'quantity': ['sum', 'mean', 'count'],
        'gross_value': ['sum', 'mean'],
        'net_value': ['sum', 'mean'],
        'gross_profit': ['sum', 'mean']
    }).reset_index()
    
    weekly_data.columns = ['_'.join(col).strip() if col[1] else col[0] for col in weekly_data.columns.values]
    
    column_mapping = {
        'quantity_sum': 'total_quantity',
        'quantity_mean': 'avg_quantity_per_transaction',
        'quantity_count': 'num_transactions',
        'gross_value_sum': 'total_gross_value',
        'gross_value_mean': 'avg_gross_value',
        'net_value_sum': 'total_net_value',
        'net_value_mean': 'avg_net_value',
        'gross_profit_sum': 'total_gross_profit',
        'gross_profit_mean': 'avg_gross_profit'
    }
    weekly_data.rename(columns=column_mapping, inplace=True)
    
    self.weekly_data = weekly_data
    print(f"Created weekly aggregations: {len(self.weekly_data):,} rows")
    
    # Cap weekly quantity outliers per store-product pair
    print("Capping weekly quantity outliers...")
    self.cap_weekly_outliers()

SalesForecastModelV2.create_weekly_aggregations = create_weekly_aggregations


In [7]:
# Cap extreme weekly quantities using robust per-group statistics
def cap_weekly_outliers(self):
    # Calculate caps per store-product pair (99.5th percentile)
    store_product_caps = self.weekly_data.groupby(['internal_store_id', 'internal_product_id'])['total_quantity'].quantile(0.995).reset_index()
    store_product_caps.rename(columns={'total_quantity': 'sp_quantity_cap'}, inplace=True)
    
    # Calculate global cap as backup (99.8th percentile)
    global_cap = self.weekly_data['total_quantity'].quantile(0.998)
    
    # Merge caps
    self.weekly_data = self.weekly_data.merge(
        store_product_caps, 
        on=['internal_store_id', 'internal_product_id'], 
        how='left'
    )
    
    # Apply caps (use store-product cap, fallback to global cap)
    before_sum = self.weekly_data['total_quantity'].sum()
    before_max = self.weekly_data['total_quantity'].max()
    
    self.weekly_data['sp_quantity_cap'] = self.weekly_data['sp_quantity_cap'].fillna(global_cap)
    self.weekly_data['total_quantity'] = np.minimum(
        self.weekly_data['total_quantity'], 
        self.weekly_data['sp_quantity_cap']
    )
    
    after_sum = self.weekly_data['total_quantity'].sum()
    after_max = self.weekly_data['total_quantity'].max()
    
    # Clean up temporary column
    self.weekly_data.drop('sp_quantity_cap', axis=1, inplace=True)
    
    print(f"  Before capping: max={before_max:,.0f}, total={before_sum:,.0f}")
    print(f"  After capping:  max={after_max:,.0f}, total={after_sum:,.0f}")
    print(f"  Reduction: {((before_sum - after_sum)/before_sum)*100:.2f}% of total volume capped")

SalesForecastModelV2.cap_weekly_outliers = cap_weekly_outliers


In [8]:
# Build enhanced features including temporal, lag, trend, and interaction features
def build_features_v2(self):
    print("\nBuilding enhanced features...")
    
    self.weekly_data = self.weekly_data.sort_values(['internal_store_id', 'internal_product_id', 'week'])
    
    print("Creating temporal features...")
    self.weekly_data['week_sin'] = np.sin(2 * np.pi * self.weekly_data['week'] / 52)
    self.weekly_data['week_cos'] = np.cos(2 * np.pi * self.weekly_data['week'] / 52)
    self.weekly_data['month_sin'] = np.sin(2 * np.pi * self.weekly_data['month'] / 12)
    self.weekly_data['month_cos'] = np.cos(2 * np.pi * self.weekly_data['month'] / 12)
    
    print("Creating trend features...")
    grouped = self.weekly_data.groupby(['internal_store_id', 'internal_product_id'])
    
    self.weekly_data['quantity_lag_1'] = grouped['total_quantity'].shift(1)
    self.weekly_data['quantity_lag_2'] = grouped['total_quantity'].shift(2)
    
    for window in [2, 4]:
        past_total = grouped['total_quantity'].shift(1)
        self.weekly_data[f'quantity_rolling_avg_{window}'] = past_total.rolling(window=window, min_periods=1).mean()
    
    self.weekly_data['quantity_trend_2w'] = (self.weekly_data['quantity_lag_1'] - self.weekly_data['quantity_lag_2']) / (self.weekly_data['quantity_lag_2'] + 1)
    self.weekly_data['quantity_lag_3'] = grouped['total_quantity'].shift(3)
    self.weekly_data['quantity_growth_rate'] = (self.weekly_data['quantity_lag_1'] - self.weekly_data['quantity_lag_3']) / (self.weekly_data['quantity_lag_3'] + 1)
    
    print("Creating lifecycle features...")
    first_sale = grouped['week'].transform('min')
    last_sale = grouped['week'].transform('max')
    self.weekly_data['weeks_since_first_sale'] = self.weekly_data['week'] - first_sale
    self.weekly_data['weeks_since_last_sale'] = self.weekly_data['week'] - last_sale
    
    print("Creating store-product interaction features...")
    store_product_stats = self.weekly_data.groupby(['internal_store_id', 'internal_product_id']).agg({
        'total_quantity': ['mean', 'std', 'count'],
        'num_transactions': 'mean'
    })
    
    store_product_stats.columns = ['_'.join(col).strip() if col[1] else col[0] for col in store_product_stats.columns.values]
    store_product_stats.rename(columns={
        'total_quantity_mean': 'store_product_avg_quantity',
        'total_quantity_std': 'store_product_std_quantity',
        'total_quantity_count': 'store_product_weeks_active',
        'num_transactions_mean': 'store_product_avg_transactions'
    }, inplace=True)
    
    self.weekly_data = self.weekly_data.merge(
        store_product_stats,
        left_on=['internal_store_id', 'internal_product_id'],
        right_index=True,
        how='left'
    )
    
    print("Creating category performance features...")
    self.weekly_data = self.weekly_data.sort_values(['categoria', 'week'])
    self.weekly_data['category_weekly_avg'] = (
        self.weekly_data.groupby('categoria')['total_quantity']
        .apply(lambda s: s.shift(1).expanding(min_periods=1).mean())
        .reset_index(level=0, drop=True)
    )
    
    self.weekly_data = self.weekly_data.sort_values(['internal_store_id', 'week'])
    self.weekly_data['store_weekly_avg'] = (
        self.weekly_data.groupby('internal_store_id')['total_quantity']
        .apply(lambda s: s.shift(1).expanding(min_periods=1).mean())
        .reset_index(level=0, drop=True)
    )
    
    self.weekly_data = self.weekly_data.fillna(0)
    print("Enhanced feature engineering completed")

SalesForecastModelV2.build_features_v2 = build_features_v2


In [9]:
# Prepare training data with feature selection and target transformation
def prepare_training_data_v2(self):
    print("\nPreparing training data...")
    
    categorical_features = ['categoria', 'marca', 'premise', 'categoria_pdv']
    
    # Keep original categorical columns for CatBoost native handling
    for feature in categorical_features:
        self.weekly_data[feature] = self.weekly_data[feature].astype(str)
    
    feature_columns = [
        'week', 'month', 'quarter',
        'week_sin', 'week_cos', 'month_sin', 'month_cos',
        'num_transactions', 'avg_quantity_per_transaction',
        'total_gross_value', 'avg_gross_value',
        'total_net_value', 'avg_net_value',
        'total_gross_profit', 'avg_gross_profit',
        'quantity_lag_1', 'quantity_lag_2', 'quantity_lag_3',
        'quantity_rolling_avg_2', 'quantity_rolling_avg_4',
        'quantity_trend_2w', 'quantity_growth_rate',
        'weeks_since_first_sale', 'weeks_since_last_sale',
        'store_product_avg_quantity', 'store_product_std_quantity',
        'store_product_weeks_active', 'store_product_avg_transactions',
        'category_weekly_avg', 'store_weekly_avg'
    ] + categorical_features
    
    self.categorical_features = categorical_features
    
    train_data = self.weekly_data[self.weekly_data['week'] >= 5].copy()
    
    X = train_data[feature_columns]
    y = train_data['total_quantity']
    
    if self.target_log_transformed:
        y = np.log1p(y)
        print("Applied log1p transformation to target")
    
    print(f"Training data shape: X={X.shape}, y={y.shape}")
    
    return X, y, feature_columns, train_data

SalesForecastModelV2.prepare_training_data_v2 = prepare_training_data_v2


In [10]:
# Calculate Weighted Mean Absolute Percentage Error (WMAPE) and create time-series split
def calculate_wmape(self, y_true, y_pred):
    if self.target_log_transformed:
        y_true = np.expm1(y_true)
        y_pred = np.expm1(y_pred)
    y_pred = np.maximum(0, y_pred)
    denom = np.sum(y_true)
    return 0.0 if denom == 0 else np.sum(np.abs(y_true - y_pred)) / denom * 100

def time_series_split(self, X, y, test_weeks=6):
    train_data = self.weekly_data[self.weekly_data['week'] >= 5].copy()
    max_week = train_data['week'].max()
    split_week = max_week - test_weeks + 1
    
    train_mask = train_data['week'] < split_week
    val_mask = train_data['week'] >= split_week
    
    X_train = X[train_mask]
    X_val = X[val_mask]
    y_train = y[train_mask]
    y_val = y[val_mask]
    
    print(f"Time-based split: Train weeks 5-{split_week-1}, Validation weeks {split_week}-{max_week}")
    print(f"Train samples: {len(X_train):,}, Validation samples: {len(X_val):,}")
    
    return X_train, X_val, y_train, y_val

SalesForecastModelV2.calculate_wmape = calculate_wmape
SalesForecastModelV2.time_series_split = time_series_split


In [11]:
# Train CatBoost model with GPU/multicore acceleration and optimized hyperparameters
def train_model_v2(self, X, y, optimize_hyperparams=False, n_trials=150, use_gpu=True):
    print("\nTraining enhanced CatBoost model...")
    
    X_train, X_val, y_train, y_val = self.time_series_split(X, y)
    
    # Check GPU availability
    gpu_available = False
    if use_gpu:
        try:
            # Try creating a small GPU model to test
            test_model = CatBoostRegressor(iterations=1, task_type="GPU", verbose=False)
            test_model.fit(X_train.head(100), y_train.head(100), verbose=False)
            gpu_available = True
            print("GPU acceleration available and enabled")
        except Exception as e:
            print(f"GPU not available: {str(e)}")
            gpu_available = False
    
    # Get optimal thread count for CPU
    import os
    n_threads = os.cpu_count()
    print(f"🔧 Using {n_threads} CPU threads")
    
    model_params = {
        'iterations': 1500,
        'learning_rate': 0.03,
        'depth': 8,
        'l2_leaf_reg': 15,
        'bagging_temperature': 0.8,
        'random_strength': 1.5,
        'border_count': 200,
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': 42,
        'verbose': 100,
        'early_stopping_rounds': 200,
        'thread_count': n_threads,  # Use all available CPU cores
    }
    
    # Add GPU-specific parameters if available
    if gpu_available and use_gpu:
        model_params.update({
            'task_type': 'GPU',
            'devices': '0',  # Use first GPU
            'gpu_ram_part': 0.5,  # Use 50% of GPU memory
        })
        print("GPU acceleration enabled")
    else:
        print("Using CPU training with multicore support")
    
    print(f"Using parameters: {model_params}")
    self.model = CatBoostRegressor(**model_params)
    
    cat_feature_indices = [X_train.columns.get_loc(col) for col in self.categorical_features if col in X_train.columns]
    
    self.model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_feature_indices,
        early_stopping_rounds=150,
        verbose=100,
        use_best_model=True
    )
    
    self.evaluate_model_v2(X_train, X_val, y_train, y_val)

SalesForecastModelV2.train_model_v2 = train_model_v2


In [12]:
# Evaluate model performance on training and validation sets
def evaluate_model_v2(self, X_train, X_val, y_train, y_val):
    train_pred = self.model.predict(X_train)
    val_pred = self.model.predict(X_val)
    
    train_wmape = self.calculate_wmape(y_train, train_pred)
    val_wmape = self.calculate_wmape(y_val, val_pred)
    
    if self.target_log_transformed:
        y_train_orig = np.expm1(y_train)
        y_val_orig = np.expm1(y_val)
        train_pred_orig = np.expm1(train_pred)
        val_pred_orig = np.expm1(val_pred)
    else:
        y_train_orig = y_train
        y_val_orig = y_val
        train_pred_orig = train_pred
        val_pred_orig = val_pred
    
    train_pred_orig = np.maximum(0, train_pred_orig)
    val_pred_orig = np.maximum(0, val_pred_orig)
    
    train_mape = mean_absolute_percentage_error(y_train_orig, train_pred_orig) * 100
    val_mape = mean_absolute_percentage_error(y_val_orig, val_pred_orig) * 100
    
    self.validation_metrics = {
        'train_mape': train_mape,
        'val_mape': val_mape,
        'train_wmape': train_wmape,
        'val_wmape': val_wmape,
        'train_samples': len(y_train),
        'val_samples': len(y_val)
    }
    
    print(f"Training MAPE: {train_mape:.2f}%")
    print(f"Validation MAPE: {val_mape:.2f}%")
    print(f"Training WMAPE: {train_wmape:.2f}%")
    print(f"Validation WMAPE: {val_wmape:.2f}%")
    
    return self.validation_metrics

SalesForecastModelV2.evaluate_model_v2 = evaluate_model_v2


In [13]:
# Train CatBoost model optimized for Apple M4 with maximum CPU performance
def train_model_v2(self, X, y, optimize_hyperparams=False, n_trials=150, use_gpu=True):
    print("\nTraining enhanced CatBoost model...")
    
    X_train, X_val, y_train, y_val = self.time_series_split(X, y)
    
    # System detection and optimization
    import platform
    import os
    system_info = platform.system()
    machine_info = platform.machine()
    print(f"System: {system_info} {machine_info}")
    
    # GPU availability check (CatBoost supports NVIDIA CUDA only)
    gpu_available = False
    if use_gpu:
        try:
            test_model = CatBoostRegressor(iterations=1, task_type="GPU", verbose=False)
            test_model.fit(X_train.head(100), y_train.head(100), verbose=False)
            gpu_available = True
            print("NVIDIA GPU acceleration enabled")
        except Exception:
            print("CUDA GPU not available - using optimized CPU training")
            print("Note: Apple Silicon GPU cores not supported by CatBoost")
            gpu_available = False
    
    # Thread optimization for Apple Silicon
    n_threads = os.cpu_count()
    if system_info == "Darwin" and ("arm64" in machine_info or "arm" in machine_info):
        print("Apple Silicon detected - optimizing thread allocation")
        n_threads = max(1, int(n_threads * 0.9))  # Reserve some cores for system
    
    print(f"Using {n_threads} CPU threads")
    
    # Optimized model parameters for Apple Silicon
    model_params = {
        'iterations': 1500,
        'learning_rate': 0.03,
        'depth': 8,
        'l2_leaf_reg': 15,
        'bootstrap_type': 'Bayesian',  # Fixed: Compatible with bagging_temperature
        'bagging_temperature': 0.8,
        'random_strength': 1.5,
        'border_count': 200,
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': 42,
        'verbose': 100,
        'early_stopping_rounds': 200,
        'thread_count': n_threads,
        'used_ram_limit': '12GB',  # Increased for M4 systems
    }
    
    # GPU parameters if available
    if gpu_available and use_gpu:
        model_params.update({
            'task_type': 'GPU',
            'devices': '0',
            'gpu_ram_part': 0.5,
        })
        print("GPU acceleration enabled")
    else:
        print("CPU training optimized for Apple Silicon")
    
    print(f"Model parameters: {model_params}")
    self.model = CatBoostRegressor(**model_params)
    
    cat_feature_indices = [X_train.columns.get_loc(col) for col in self.categorical_features if col in X_train.columns]
    
    print("Starting model training...")
    self.model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_feature_indices,
        early_stopping_rounds=150,
        verbose=100,
        use_best_model=True
    )
    
    self.evaluate_model_v2(X_train, X_val, y_train, y_val)

SalesForecastModelV2.train_model_v2 = train_model_v2

In [14]:
# Analyze prediction distribution, save predictions, and print performance report
def analyze_predictions(self, predictions_df):
    print("\nPrediction Analysis:")
    print(f"Total predictions: {len(predictions_df):,}")
    print(f"Zero predictions: {(predictions_df['quantidade'] == 0).sum():,}")
    print(f"Non-zero predictions: {(predictions_df['quantidade'] > 0).sum():,}")
    print(f"Mean prediction: {predictions_df['quantidade'].mean():.2f}")
    print(f"Median prediction: {predictions_df['quantidade'].median():.2f}")
    print(f"Max prediction: {predictions_df['quantidade'].max():,}")
    print(f"Std prediction: {predictions_df['quantidade'].std():.2f}")
    
    quantiles = predictions_df['quantidade'].quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
    print("Prediction quantiles:")
    for q, val in quantiles.items():
        print(f"  {q*100:.0f}%: {val:.2f}")
    
    weekly_stats = predictions_df.groupby('semana')['quantidade'].agg(['count', 'mean', 'sum']).round(2)
    print("\nWeekly prediction summary:")
    print(weekly_stats)

def save_predictions(self, predictions_df, filename="sales_predictions_v2.csv"):
    print(f"\nSaving predictions to {filename}...")
    predictions_df.to_csv(filename, sep=';', index=False, encoding='utf-8')
    
    parquet_filename = filename.replace('.csv', '.parquet')
    predictions_df.to_parquet(parquet_filename, index=False)
    
    print(f"Predictions saved successfully!")
    print(f"CSV File: {filename}")
    print(f"Parquet File: {parquet_filename}")
    print(f"Rows: {len(predictions_df):,}")
    print(f"Sample:")
    print(predictions_df.head(10))

def print_performance_report(self):
    print("\n" + "="*50)
    print("PERFORMANCE REPORT V2")
    print("="*50)
    
    if self.validation_metrics:
        print("Model Validation Metrics:")
        print(f"  Training MAPE: {self.validation_metrics['train_mape']:.2f}%")
        print(f"  Validation MAPE: {self.validation_metrics['val_mape']:.2f}%")
        print(f"  Training WMAPE: {self.validation_metrics['train_wmape']:.2f}%")
        print(f"  Validation WMAPE: {self.validation_metrics['val_wmape']:.2f}%")
        print(f"  Training samples: {self.validation_metrics['train_samples']:,}")
        print(f"  Validation samples: {self.validation_metrics['val_samples']:,}")
        
        wmape_diff = abs(self.validation_metrics['val_wmape'] - self.validation_metrics['train_wmape'])
        mape_diff = abs(self.validation_metrics['val_mape'] - self.validation_metrics['train_mape'])
        
        print(f"\nOverfitting Check:")
        print(f"  MAPE difference: {mape_diff:.2f}%")
        print(f"  WMAPE difference: {wmape_diff:.2f}%")
        
        if wmape_diff < 2 and mape_diff < 10:
            print("  Status: Good generalization")
        elif wmape_diff < 5 and mape_diff < 20:
            print("  Status: Moderate overfitting")
        else:
            print("  Status: High overfitting risk")
    
    print("="*50)

SalesForecastModelV2.analyze_predictions = analyze_predictions
SalesForecastModelV2.save_predictions = save_predictions
SalesForecastModelV2.print_performance_report = print_performance_report


In [15]:
# Generate predictions for the next 5 weeks with enhanced feature handling
def generate_predictions_v2(self, feature_columns, train_data, max_rows=1500000, weeks_to_predict=5, recent_weeks=8):
    print("\nGenerating predictions for January 2023...")
    
    # Discover unique store-product combinations
    store_product_combinations = train_data[['internal_store_id', 'internal_product_id']].drop_duplicates()
    print(f"Found {len(store_product_combinations):,} unique store-product combinations")
    
    # Build latest snapshot per pair with all features
    latest_records = train_data.loc[train_data.groupby(['internal_store_id', 'internal_product_id'])['week'].idxmax()].copy()
    
    # Select top active pairs to respect the 1.5M-row portal limit
    try:
        pairs_limit = max_rows // weeks_to_predict
        max_week = int(self.weekly_data['week'].max())
        start_week = max(1, max_week - int(recent_weeks) + 1)
        recent_slice = self.weekly_data[self.weekly_data['week'] >= start_week]
        activity = recent_slice.groupby(['internal_store_id', 'internal_product_id']).agg(
            recent_total_qty=('total_quantity', 'sum'),
            weeks_with_sales=('total_quantity', lambda s: int((s > 0).sum())),
            last_week_seen=('week', 'max')
        ).reset_index()
        activity = activity.sort_values(
            by=['recent_total_qty', 'weeks_with_sales', 'last_week_seen'],
            ascending=[False, False, False]
        )
        selected_pairs = activity.head(pairs_limit)[['internal_store_id', 'internal_product_id']]
        before = len(latest_records)
        latest_records = latest_records.merge(selected_pairs, on=['internal_store_id', 'internal_product_id'], how='inner')
        after = len(latest_records)
        print(f"Selected top {after:,} active pairs out of {before:,} (recent_weeks={recent_weeks})")
    except Exception as e:
        print(f"Pair selection step skipped due to error: {e}")
    
    all_predictions = []
    
    for week in range(1, weeks_to_predict + 1):
        print(f"Predicting week {week}...")
        
        week_data = latest_records.copy()
        
        # Update temporal features for the prediction week
        week_data['week'] = week
        week_data['month'] = 1  # January
        week_data['quarter'] = 1  # Q1
        
        # Update seasonal features
        week_data['week_sin'] = np.sin(2 * np.pi * week / 52)
        week_data['week_cos'] = np.cos(2 * np.pi * week / 52)
        week_data['month_sin'] = np.sin(2 * np.pi * 1 / 12)  # January
        week_data['month_cos'] = np.cos(2 * np.pi * 1 / 12)  # January
        
        # Update lifecycle features
        week_data['weeks_since_last_sale'] = 0  # Assume continuing sales
        
        # Ensure all required features are present
        missing_features = [f for f in feature_columns if f not in week_data.columns]
        if missing_features:
            print(f"Warning: Missing features for prediction: {missing_features}")
            # Fill missing features with zeros or appropriate defaults
            for feature in missing_features:
                week_data[feature] = 0
        
        # Prepare feature matrix
        try:
            features_matrix = week_data[feature_columns]
            
            # Make predictions
            predictions = self.model.predict(features_matrix)
            
            # Handle log transformation if used
            if self.target_log_transformed:
                predictions = np.expm1(predictions)
            
            # Ensure non-negative predictions
            predictions = np.maximum(0, predictions)
            
            # Create prediction dataframe
            week_predictions = pd.DataFrame({
                'semana': week,
                'pdv': week_data['internal_store_id'].astype(int),
                'produto': week_data['internal_product_id'].astype(int),
                'quantidade': predictions.round().astype(int)
            })
            
            all_predictions.append(week_predictions)
            print(f"Generated {len(week_predictions):,} predictions for week {week}")
            
        except Exception as e:
            print(f"Error generating predictions for week {week}: {e}")
            # Create fallback predictions with zeros
            week_predictions = pd.DataFrame({
                'semana': week,
                'pdv': week_data['internal_store_id'].astype(int),
                'produto': week_data['internal_product_id'].astype(int),
                'quantidade': 0
            })
            all_predictions.append(week_predictions)
    
    predictions_df = pd.concat(all_predictions, ignore_index=True)
    
    # Final safeguard
    if len(predictions_df) > max_rows:
        print(f"Limiting predictions to {max_rows:,} rows (was {len(predictions_df):,})")
        predictions_df = predictions_df.head(max_rows)
    
    print(f"Total predictions generated: {len(predictions_df):,}")
    self.analyze_predictions(predictions_df)
    return predictions_df

SalesForecastModelV2.generate_predictions_v2 = generate_predictions_v2

In [16]:
# Execute the complete pipeline: data loading, processing, training, and prediction
def run_complete_pipeline(self):
    print("=== Sales Forecast Model V2 - Enhanced ===")
    
    self.load_data()
    self.cleanse_data()
    self.merge_data()
    self.create_weekly_aggregations()
    self.build_features_v2()
    
    X, y, feature_columns, train_data = self.prepare_training_data_v2()
    self.train_model_v2(X, y, optimize_hyperparams=False, n_trials=150)
    
    predictions_df = self.generate_predictions_v2(feature_columns, train_data)
    self.save_predictions(predictions_df)
    self.print_performance_report()
    
    return predictions_df

SalesForecastModelV2.run_complete_pipeline = run_complete_pipeline


In [17]:
# Initialize and run the complete sales forecasting pipeline
try:
    model = SalesForecastModelV2()
    predictions = model.run_complete_pipeline()
    
    print("\n=== V2 Pipeline completed successfully! ===")
    print(f"Generated {len(predictions):,} predictions for 5 weeks of January 2023")
    
except Exception as e:
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()


=== Sales Forecast Model V2 - Enhanced ===
Loading data...
Found 3 parquet files
part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet: Shape (6560698, 11)
-> Identified as TRANSACTIONS data
part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet: Shape (7092, 8)
-> Identified as PRODUCTS data
part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet: Shape (14419, 4)
-> Identified as STORES data
Data loaded successfully:
- Transactions: 6,560,698 rows
- Products: 7,092 rows
- Stores: 14,419 rows

Cleansing data...
Removed 0 rows with null values
Kept 6,430,161 rows with positive quantities
Removing extreme value outliers...
Removed 94,398 outlier transactions (1.47%)
Filtered to 2022 data: 6,335,763 rows
Cleaning products and stores...

Merging data...
After product merge: 6,335,763 rows
After store merge: 6,335,763 rows
Data merge completed

Creating weekly aggregat